In [4]:
import pandas as pd

from meteostat import Point, Hourly
import datetime


import altair as alt
from vega_datasets import data


In [2]:
from Tables_code import get_tornado_data, generate_tornado_map

tornado = get_tornado_data()


In [20]:
def generate_tornado_map(tornado):
    alt.data_transformers.enable("vegafusion")
    # ['tornado_number', 'state', 'magnitude', 'lat', 'lon', 'time', 'wspd', 'wdir']

    # --------------------------------
    # Tornado Locations (unique per tornado)
    # --------------------------------
    df = (
        tornado[['tornado_number', 'state', 'magnitude', 'lat', 'lon']]
        .drop_duplicates(subset=['tornado_number'])
    )

    # --------------------------------
    # HOURLY WIND DATA
    # --------------------------------
    # No grouping, keep each hourly record
    hourly_wind = tornado[['tornado_number', 'time','time_delta', 'wspd',"magnitude",'lat','lon']].copy()
    hourly_wind = hourly_wind.rename(columns={'wspd': 'wind_speed'})

    # --------------------------------
    # INTERACTIVE SELECTION
    # --------------------------------
    geographic_brush  = alt.selection_point(fields=['tornado_number'])
    
    # U.S. map base layer
    us_states = alt.topo_feature(data.us_10m.url, feature='states')

    base_map = alt.Chart(us_states).mark_geoshape(
        fill="#f0f0f0", stroke='black'
    ).project("albersUsa").properties(width=600, height=300)

    # Tornado starting points
    points = alt.Chart(df).mark_circle(size=200, opacity=0.8).encode(
        longitude='lon:Q',
        latitude='lat:Q',
        color=alt.Color('magnitude:N'),
        tooltip=['tornado_number:N', 'state:N', 'magnitude:Q'],
        opacity=alt.condition(geographic_brush, 
                          alt.value(0.8), alt.value(0.4))
    ).add_params(
        geographic_brush
    )

    map_layer = base_map + points

    # --------------------------------
    # HOURLY LINE CHART (Filtered by Selection)
    # --------------------------------
    
    wind_chart = alt.Chart(hourly_wind).mark_line(point=True).encode(
        x=alt.X('time_delta',
            title='Time from tornado start',
            axis=alt.Axis(labelExpr="floor(datum.value/24) + 'd ' + floor(datum.value%24) + 'h'"),
            scale=alt.Scale(domain=[-3*24, 3*24])
        ),
        y=alt.Y('wind_speed:Q', title='Wind Speed (m/s)'),
        color=alt.Color('magnitude:N'),
        detail='tornado_number:N',
        tooltip=['tornado_number:N', 'time:T', 'wind_speed:Q'],
        opacity=alt.value(0.8), 
    ).transform_filter(
        geographic_brush

        ).properties(
        width=650,
        height=300,
        title='Hourly Wind Speed (Filtered by Tornado Selection)'
    )

    # --------------------------------
    # FINAL DASHBOARD
    # --------------------------------
    final_chart = alt.hconcat(map_layer, wind_chart).resolve_scale()

    return final_chart

generate_tornado_map(tornado)

alt.HConcatChart(...)

In [ ]:
tornados_df = pd.read_csv("Tornado_Tracks_1950_2017_1_7964592706304725094.csv")
tornados_df = tornados_df[tornados_df["Year"]==2024]


cat_4 =  tornados_df[tornados_df["Magnitude"]==4]
cat_3 = tornados_df[tornados_df["Magnitude"]==3].sample(n=5,random_state=42)
cat_2 = tornados_df[tornados_df["Magnitude"]==2].sample(n=5,random_state=42)
cat_1 = tornados_df[tornados_df["Magnitude"]==1].sample(n=5,random_state=42)
cat_0 = tornados_df[tornados_df["Magnitude"]==0].sample(n=5,random_state=42)

case_study= pd.concat([cat_4, cat_3, cat_2, cat_1, cat_0])
print(case_study.columns)
case_study[["State Abbreviation", "Tornado Number", "Year","Month","Day", "Time","Magnitude", "Starting Latitude",	"Starting Longitude",	"Ending Latitude",	"Ending Longitude"]]




def get_tornado_weekly_wind(tornado_df: pd.DataFrame) -> pd.DataFrame:
    """
    Fetch hourly wind data (speed & direction) for a week surrounding each tornado event.
    
    Args:
        tornado_df (pd.DataFrame): DataFrame with columns:
            ["State Abbreviation", "Tornado Number", "Year", "Month", "Day", "Time",
             "Magnitude", "Starting Latitude", "Starting Longitude", 
             "Ending Latitude", "Ending Longitude"]
             
    Returns:
        pd.DataFrame: Combined DataFrame with columns:
            ['tornado_number', 'state', 'magnitude', 'lat', 'lon', 
             'time', 'wspd', 'wdir']
    """

    all_data = []

    for idx, row in tornado_df.iterrows():
        try:
            # Parse tornado info
            tornado_num = row["Tornado Number"]
            state = row["State Abbreviation"]
            magnitude = row["Magnitude"]
            lat = row["Starting Latitude"]
            lon = row["Starting Longitude"]
            
            # Parse date and time (safe fallback)
            try:
                tornado_time = datetime.datetime(
                    int(row["Year"]),
                    int(row["Month"]),
                    int(row["Day"])
                )
            except Exception:
                print(f"⚠️ Skipping tornado {tornado_num}: Invalid date")
                continue

            # Define ±3 days around event
            start = tornado_time - datetime.timedelta(days=3)
            end = tornado_time + datetime.timedelta(days=3)

            print(f"Fetching wind for Tornado {tornado_num} ({state}) from {start.date()} to {end.date()}")

            # Create a Meteostat point and fetch hourly data
            point = Point(lat, lon)
            data = Hourly(point, start, end).fetch()

            if not data.empty:
                data = data.reset_index()
                data["tornado_number"] = tornado_num
                data["state"] = state
                data["magnitude"] = magnitude
                data["lat"] = lat
                data["lon"] = lon

                # Keep only relevant columns
                data = data[[
                    "tornado_number", "state", "magnitude", "lat", "lon",
                    "time", "wspd", "wdir"
                ]]
                all_data.append(data)
            else:
                print(f"⚠️ No wind data found for Tornado {tornado_num} ({state})")

        except Exception as e:
            print(f"❌ Error fetching Tornado {tornado_num}: {e}")
            continue

    if not all_data:
        print("No data retrieved for any tornado events.")
        return pd.DataFrame(columns=[
            "tornado_number", "state", "magnitude", "lat", "lon", "time", "wspd", "wdir"
        ])

    result = pd.concat(all_data, ignore_index=True)
    result["day_of_year"] = result["time"].dt.day_of_year

    return result
tornado = get_tornado_weekly_wind(case_study)

Index(['State Abbreviation', 'State FIPS', 'Date', 'Year', 'Month', 'Day',
       'Date (Year, Month, Day)', 'Time', 'Time Zone', 'Magnitude',
       'Width (yards)', 'Length (miles)', 'Fatalities', 'Injuries',
       'Crop Loss', 'Property Loss', 'Tornado Number', 'State Number',
       'EF Scale (unaltered or previous rating)', 'Starting Latitude',
       'Starting Longitude', 'Ending Latitude', 'Ending Longitude',
       'Object ID', 'Shape Length'],
      dtype='object')
Fetching wind for Tornado 623761 (NE) from 2024-04-23 to 2024-04-29
Fetching wind for Tornado 623863 (OK) from 2024-04-24 to 2024-04-30
Fetching wind for Tornado 624208 (IA) from 2024-05-18 to 2024-05-24
Fetching wind for Tornado 623960 (OK) from 2024-05-03 to 2024-05-09
Fetching wind for Tornado 624082 (AL) from 2024-05-05 to 2024-05-11
Fetching wind for Tornado 624328 (TX) from 2024-05-22 to 2024-05-28
Fetching wind for Tornado 624333 (OK) from 2024-05-22 to 2024-05-28
Fetching wind for Tornado 624057 (AL) from 2

In [ ]:
cities = {

    'New York, NY': (40.7128, -74.0060),
    'Boston, MA': (42.3601, -71.0589),
    'Philadelphia, PA': (39.9526, -75.1652),
    'Pittsburgh, PA': (40.4406, -79.9959),
    'Buffalo, NY': (42.8864, -78.8784),

    'Miami, FL': (25.7617, -80.1918),
    'Atlanta, GA': (33.7490, -84.3880),
    'Charlotte, NC': (35.2271, -80.8431),
    'Jacksonville, FL': (30.3322, -81.6557),
    'Tampa, FL': (27.9506, -82.4572),

    'Chicago, IL': (41.8781, -87.6298),
    'Detroit, MI': (42.3314, -83.0458),
    'Minneapolis, MN': (44.9778, -93.2650),
    'Cleveland, OH': (41.4993, -81.6944),
    'Milwaukee, WI': (43.0389, -87.9065),

    'Los Angeles, CA': (34.0522, -118.2437),
    'San Francisco, CA': (37.7749, -122.4194),
    'Seattle, WA': (47.6062, -122.3321),
    'Portland, OR': (45.5152, -122.6784),
    'Denver, CO': (39.7392, -104.9903)
}

time_range = [datetime.datetime(2024, 1, 1), datetime.datetime(2024, 12, 31)]

def get_hourly_wind(cities: dict, start: datetime.datetime, end: datetime.datetime) -> pd.DataFrame:
    """
    Fetch hourly wind data (speed & direction) for multiple cities.
    
    Args:
        cities (dict): Mapping of city names to (lat, lon) tuples.
        start (datetime): Start date for data.
        end (datetime): End date for data.
    
    Returns:
        pd.DataFrame: Combined DataFrame with columns:
                      ['city', 'time', 'wspd', 'wdir']
    """
    all_data = []

    for city, (lat, lon) in cities.items():
        try:
            print(f"Fetching: {city} ({lat}, {lon})")
            point = Point(lat, lon)
            data = Hourly(point, start, end).fetch()

            if not data.empty:
                data = data.reset_index()
                data['city'] = city
                # keep only relevant columns
                data = data[['city', 'time', 'wspd', 'wdir']]
                all_data.append(data)
            else:
                print(f"⚠️ No data for {city}")

        except Exception as e:
            print(f"❌ Error fetching {city}: {e}")
            continue

    if not all_data:
        print("No data retrieved for any city.")
        return pd.DataFrame(columns=['city', 'time', 'wspd', 'wdir'])

    return pd.concat(all_data, ignore_index=True)

wind_df = get_hourly_wind(cities=cities,start=time_range[0], end=time_range[1] )
wind_df["day_of_year"]= wind_df["time"].dt.day_of_year


Fetching: New York, NY (40.7128, -74.006)
Fetching: Boston, MA (42.3601, -71.0589)
Fetching: Philadelphia, PA (39.9526, -75.1652)
Fetching: Pittsburgh, PA (40.4406, -79.9959)
Fetching: Buffalo, NY (42.8864, -78.8784)
Fetching: Miami, FL (25.7617, -80.1918)
Fetching: Atlanta, GA (33.749, -84.388)
Fetching: Charlotte, NC (35.2271, -80.8431)
Fetching: Jacksonville, FL (30.3322, -81.6557)
Fetching: Tampa, FL (27.9506, -82.4572)
Fetching: Chicago, IL (41.8781, -87.6298)
Fetching: Detroit, MI (42.3314, -83.0458)
Fetching: Minneapolis, MN (44.9778, -93.265)
Fetching: Cleveland, OH (41.4993, -81.6944)
Fetching: Milwaukee, WI (43.0389, -87.9065)
Fetching: Los Angeles, CA (34.0522, -118.2437)
Fetching: San Francisco, CA (37.7749, -122.4194)
Fetching: Seattle, WA (47.6062, -122.3321)
Fetching: Portland, OR (45.5152, -122.6784)
Fetching: Denver, CO (39.7392, -104.9903)


In [ ]:
alt.data_transformers.enable("vegafusion")
df = pd.DataFrame([
{'city': name, 'lat': coords[0], 'lon': coords[1]}
for name, coords in cities.items()
])

time_range = [datetime.datetime(2024, 1, 1), datetime.datetime(2024, 12, 31)]


daily_avg = (
    wind_df.groupby(['city', 'day_of_year'], as_index=False)['wspd']
    .mean()
    .rename(columns={'wspd': 'wind_speed'})
)
print(daily_avg)
#print(data)


brush = alt.selection_point(fields=['city'])

# World map
world = alt.topo_feature(data.us_10m.url, feature='states')
base_map = alt.Chart(world).mark_geoshape(
    fill="#f0f0f0", stroke='black'
).project("albersUsa").properties(width=900, height=450)

# City points with selection
points = alt.Chart(df).mark_circle(size=200, opacity=0.8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color='city:N',
    tooltip=['city:N']
).add_params(
    brush
)

map_layer = base_map + points

# Wind line chart filtered by selected cities
wind_chart = alt.Chart(daily_avg).mark_line().encode(
    x='day_of_year:Q',
    y='wind_speed:Q',
    color='city:N',
    tooltip=['city:N', 'day_of_year:Q', 'wind_speed:Q']
).transform_filter(
    brush
).properties(
    width=900,
    height=300,
    title='Daily Average Wind Speed by City (Filtered by Map Selection)'
)

final_chart = alt.vconcat(map_layer, wind_chart)
final_chart

             city  day_of_year  wind_speed
0     Atlanta, GA            1   10.070833
1     Atlanta, GA            2    5.116667
2     Atlanta, GA            3    1.158333
3     Atlanta, GA            4    6.983333
4     Atlanta, GA            5       8.075
...           ...          ...         ...
7315    Tampa, FL          362   12.104167
7316    Tampa, FL          363   15.704167
7317    Tampa, FL          364        19.2
7318    Tampa, FL          365    7.916667
7319    Tampa, FL          366         8.6

[7320 rows x 3 columns]


alt.VConcatChart(...)

In [ ]:
alt.data_transformers.enable("vegafusion")

# Assume tornado DataFrame has columns:
# ['tornado_number', 'state', 'magnitude', 'lat', 'lon', 'time', 'wspd', 'wdir']

# --------------------------------
# Tornado Locations (unique per tornado)
# --------------------------------
df = (
    tornado[['tornado_number', 'state', 'magnitude', 'lat', 'lon']]
    .drop_duplicates(subset=['tornado_number'])
)

# --------------------------------
# HOURLY WIND DATA
# --------------------------------
# No grouping, keep each hourly record
hourly_wind = tornado[['tornado_number', 'time', 'wspd']].copy()
hourly_wind = hourly_wind.rename(columns={'wspd': 'wind_speed'})

# --------------------------------
# INTERACTIVE SELECTION
# --------------------------------
brush = alt.selection_point(fields=['tornado_number'])

# U.S. map base layer
us_states = alt.topo_feature(data.us_10m.url, feature='states')

base_map = alt.Chart(us_states).mark_geoshape(
    fill="#f0f0f0", stroke='black'
).project("albersUsa").properties(width=900, height=450)

# Tornado starting points
points = alt.Chart(df).mark_circle(size=200, opacity=0.8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color=alt.Color('magnitude:N'),
    tooltip=['tornado_number:N', 'state:N', 'magnitude:Q']
).add_params(
    brush
)

map_layer = base_map + points

# --------------------------------
# HOURLY LINE CHART (Filtered by Selection)
# --------------------------------
wind_chart = alt.Chart(hourly_wind).mark_line(point=True).encode(
    x=alt.X('time:T', title='Time'),
    y=alt.Y('wind_speed:Q', title='Wind Speed (m/s)'),
    color='tornado_number:N',
    tooltip=['tornado_number:N', 'time:T', 'wind_speed:Q']
).transform_filter(
    brush
).properties(
    width=900,
    height=300,
    title='Hourly Wind Speed (Filtered by Tornado Selection)'
)

# --------------------------------
# FINAL DASHBOARD
# --------------------------------
final_chart = alt.vconcat(map_layer, wind_chart).resolve_scale(color='independent')

final_chart

alt.VConcatChart(...)

In [ ]:
tornado

,tornado_number,state,magnitude,lat,lon,time,wspd,wdir,day_of_year
0,623761,NE,4,41.2000,-96.3300,2024-04-23 00:00:00,14.8,260.0,114
1,623761,NE,4,41.2000,-96.3300,2024-04-23 01:00:00,9.4,340.0,114
2,623761,NE,4,41.2000,-96.3300,2024-04-23 02:00:00,0.0,0.0,114
3,623761,NE,4,41.2000,-96.3300,2024-04-23 03:00:00,7.6,340.0,114
4,623761,NE,4,41.2000,-96.3300,2024-04-23 04:00:00,0.0,0.0,114
...,...,...,...,...,...,...,...,...,...
3185,623979,IA,0,41.0079,-93.6118,2024-05-08 20:00:00,18.4,160.0,129
3186,623979,IA,0,41.0079,-93.6118,2024-05-08 21:00:00,14.8,140.0,129
3187,623979,IA,0,41.0079,-93.6118,2024-05-08 22:00:00,14.8,100.0,129
3188,623979,IA,0,41.0079,-93.6118,2024-05-08 23:00:00,9.4,120.0,129
